In [25]:
import pandas as pd

In [26]:
url = "http://us-central1-raccoon-bi.cloudfunctions.net/psel_de_shows"
tabela_show = pd.read_json(url, orient='index')

display(tabela_show.head())

,ano,dia,mes
Nascent Letter,2020,2,1
Symbolic Toy,2020,3,1
Terrestrial Chair,2020,1,1


In [27]:
url = "http://us-central1-raccoon-bi.cloudfunctions.net/psel_de_ingressos"
tabela_ingressos = pd.read_json(url)

display(tabela_ingressos.head())

,ano,dia,mes,nome,status,tipo
0,2020,1,1,Benjamin Mcdonald,Nao Concluido,VIP
1,2020,1,1,Mitchell Kostyla,Nao Concluido,VIP
2,2020,1,1,Gerald Allen,Nao Concluido,VIP
3,2020,1,1,Frederick Bank,Problema no Pagamento,VIP
4,2020,1,1,Tina Yee,Nao Concluido,VIP


In [28]:
url = "http://us-central1-raccoon-bi.cloudfunctions.net/psel_de_compras"
tabela_compras = pd.read_csv(url)

tabela_compras.head()

,nome,show,gastos
0,Joseph Wray,Symbolic Toy,3142.35
1,Barbara Mcneil,Symbolic Toy,2431.78
2,Eduardo Espenshade,Symbolic Toy,386.32
3,Lucia Hayes,Symbolic Toy,3373.49
4,Johnny Hairston,Symbolic Toy,2805.88


In [29]:
tabela_ingressos['status'].unique()

array(['Nao Concluido', 'Problema no Pagamento', 'Concluido'],
      dtype=object)

In [30]:
tabela_ingressos['tipo'].unique()

array(['VIP', 'Pista', 'Pista Premium'], dtype=object)

In [31]:
tabela_show.index.name = 'show'
tabela_show

,ano,dia,mes
show,,,
Nascent Letter,2020,2,1
Symbolic Toy,2020,3,1
Terrestrial Chair,2020,1,1


In [32]:
join_compras_show = tabela_show.join(tabela_compras.set_index('show'), how='outer')
join_compras_show.head()

,ano,dia,mes,nome,gastos
show,,,,,
Nascent Letter,2020,2,1,Kathleen Duncan,2211.61
Nascent Letter,2020,2,1,Guadalupe Bell,2420.44
Nascent Letter,2020,2,1,Margaret Robinson,3148.62
Nascent Letter,2020,2,1,Shayla Baird,1613.10
Nascent Letter,2020,2,1,Peggy Baughman,138.26


In [33]:


tabela_compras.head()

,nome,show,gastos
0,Joseph Wray,Symbolic Toy,3142.35
1,Barbara Mcneil,Symbolic Toy,2431.78
2,Eduardo Espenshade,Symbolic Toy,386.32
3,Lucia Hayes,Symbolic Toy,3373.49
4,Johnny Hairston,Symbolic Toy,2805.88


In [34]:


tabela_compras.set_index('show').head()

,nome,gastos
show,,
Symbolic Toy,Joseph Wray,3142.35
Symbolic Toy,Barbara Mcneil,2431.78
Symbolic Toy,Eduardo Espenshade,386.32
Symbolic Toy,Lucia Hayes,3373.49
Symbolic Toy,Johnny Hairston,2805.88


In [35]:
key_join_all = ['ano', 'dia', 'mes', 'nome']
join_all = join_compras_show.reset_index().set_index(key_join_all).join(tabela_ingressos.set_index(key_join_all), how='outer'
                                                         ).reset_index()
join_all.head()

,ano,dia,mes,nome,show,gastos,status,tipo
0,2020,1,1,Adelina Thews,Terrestrial Chair,2372.34,NaN,NaN
1,2020,1,1,Albert Fadden,Terrestrial Chair,1637.19,NaN,NaN
2,2020,1,1,Alberto Gumm,Terrestrial Chair,2297.47,NaN,NaN
3,2020,1,1,Allen Bylsma,Terrestrial Chair,348.84,NaN,NaN
4,2020,1,1,Andrea Galyen,Terrestrial Chair,3467.13,Nao Concluido,Pista


In [36]:
join_all = join_all.drop('show', 
                         axis=1).set_index(['ano', 'dia', 'mes']
                                          ).join(tabela_show.reset_index().set_index(['ano', 'dia', 'mes'])
                                                         ).reset_index()
join_all.head()

,ano,dia,mes,nome,gastos,status,tipo,show
0,2020,1,1,Adelina Thews,2372.34,NaN,NaN,Terrestrial Chair
1,2020,1,1,Albert Fadden,1637.19,NaN,NaN,Terrestrial Chair
2,2020,1,1,Alberto Gumm,2297.47,NaN,NaN,Terrestrial Chair
3,2020,1,1,Allen Bylsma,348.84,NaN,NaN,Terrestrial Chair
4,2020,1,1,Andrea Galyen,3467.13,Nao Concluido,Pista,Terrestrial Chair


# 1.

> Qual a média de gastos de pessoas com ingresso Pista?

Considerando a média de cada pessoa por show (i.e.: Pessoas que foram a mais de um show, contarão cada uma das vezes separada)

In [37]:
# Procurar pessoas com ingresso pista
# Calcular média do gasto dessas pessoas
pessoas_pista = join_all[(join_all['tipo'] == 'Pista') & (join_all['status'] == 'Concluido') & (pd.notna(join_all['gastos']))]
pessoas_pista.head()

,ano,dia,mes,nome,gastos,status,tipo,show
47,2020,1,1,Donna Wal,2841.61,Concluido,Pista,Terrestrial Chair
75,2020,1,1,Gretchen Hinojosa,2867.60,Concluido,Pista,Terrestrial Chair
271,2020,2,1,John Rich,2411.30,Concluido,Pista,Nascent Letter
275,2020,2,1,John Rich,1331.66,Concluido,Pista,Nascent Letter
279,2020,2,1,John Rich,421.03,Concluido,Pista,Nascent Letter


In [38]:
pessoas_pista['gastos'].mean()

2214.6885714285713

# 2. 
> Quais pessoas não compareceram aos shows?

In [39]:
# df.loc[linhas, colunas]

join_all.loc[pd.isna(join_all['gastos']), 'nome'].unique()

array(['Brenda Key', 'Courtney Edwards', 'Deborah Silsby',
       'Donna Chandler', 'Elizabeth Daly', 'Frank Houck', 'James Reaves',
       'Maria Villa', 'Nancy Smith', 'Pamela Nixon', 'Ricardo Clasby',
       'Richard Culpepper', 'Travis Whatley', 'Alice Gamba', 'Amy Mejia',
       'Frances Lambert', 'James Coffey', 'Linda Batiz', 'Matt Smudrick',
       'Rick Fleishman', 'Brenda Kozak', 'Curtis Hernandez',
       'Edward Smith', 'Marcela Nelson', 'Miguel Ramey', 'Nancy Degroot',
       'Robert Copenhaver', 'Sandra Duncan', 'Yong Baltazar'],
      dtype=object)

# 3.
> Quais pessoas compraram ingressos com concorrentes?

In [40]:
compra_concluida = join_all[(join_all['status'] == 'Concluido')]
compra_concluida.head()

,ano,dia,mes,nome,gastos,status,tipo,show
14,2020,1,1,Brenda Key,NaN,Concluido,Pista Premium,Terrestrial Chair
22,2020,1,1,Clara Faupel,2968.47,Concluido,Pista Premium,Terrestrial Chair
25,2020,1,1,Courtney Edwards,NaN,Concluido,Pista Premium,Terrestrial Chair
32,2020,1,1,Deborah Silsby,NaN,Concluido,VIP,Terrestrial Chair
47,2020,1,1,Donna Wal,2841.61,Concluido,Pista,Terrestrial Chair


In [41]:
gastos_pessoas = join_all[pd.notna(join_all['gastos'])]
gastos_pessoas.head()

,ano,dia,mes,nome,gastos,status,tipo,show
0,2020,1,1,Adelina Thews,2372.34,NaN,NaN,Terrestrial Chair
1,2020,1,1,Albert Fadden,1637.19,NaN,NaN,Terrestrial Chair
2,2020,1,1,Alberto Gumm,2297.47,NaN,NaN,Terrestrial Chair
3,2020,1,1,Allen Bylsma,348.84,NaN,NaN,Terrestrial Chair
4,2020,1,1,Andrea Galyen,3467.13,Nao Concluido,Pista,Terrestrial Chair


In [42]:
# Pessoas que tiveram gasto num dia, mas que NÃO compraram ingresso com sucesso para aquele show


join_gastos_concluido = gastos_pessoas.set_index(['nome', 'show']).join(compra_concluida.set_index(['nome', 'show']), how='outer', lsuffix='_l')
join_gastos_concluido.head()

ano_l  dia_l  mes_l  gastos_l status_l  \
nome          show                                                         
Adelina Thews Nascent Letter     2020.0    2.0    1.0    371.39      NaN   
              Terrestrial Chair  2020.0    1.0    1.0   2372.34      NaN   
Alan Gravois  Nascent Letter     2020.0    2.0    1.0    463.15      NaN   
Albert Fadden Terrestrial Chair  2020.0    1.0    1.0   1637.19      NaN   
Alberto Gumm  Terrestrial Chair  2020.0    1.0    1.0   2297.47      NaN   

                                tipo_l  ano  dia  mes  gastos status tipo  
nome          show                                                         
Adelina Thews Nascent Letter       NaN  NaN  NaN  NaN     NaN    NaN  NaN  
              Terrestrial Chair    NaN  NaN  NaN  NaN     NaN    NaN  NaN  
Alan Gravois  Nascent Letter       NaN  NaN  NaN  NaN     NaN    NaN  NaN  
Albert Fadden Terrestrial Chair    NaN  NaN  NaN  NaN     NaN    NaN  NaN  
Alberto Gumm  Terrestrial Chair    NaN  NaN  NaN  NaN     NaN    NaN  NaN

In [43]:
gasto_sem_compra_concluida = join_gastos_concluido[
    pd.notna(join_gastos_concluido['ano_l']) & pd.isna(join_gastos_concluido['ano'])]
gasto_sem_compra_concluida.head()

ano_l  dia_l  mes_l  gastos_l status_l  \
nome          show                                                         
Adelina Thews Nascent Letter     2020.0    2.0    1.0    371.39      NaN   
              Terrestrial Chair  2020.0    1.0    1.0   2372.34      NaN   
Alan Gravois  Nascent Letter     2020.0    2.0    1.0    463.15      NaN   
Albert Fadden Terrestrial Chair  2020.0    1.0    1.0   1637.19      NaN   
Alberto Gumm  Terrestrial Chair  2020.0    1.0    1.0   2297.47      NaN   

                                tipo_l  ano  dia  mes  gastos status tipo  
nome          show                                                         
Adelina Thews Nascent Letter       NaN  NaN  NaN  NaN     NaN    NaN  NaN  
              Terrestrial Chair    NaN  NaN  NaN  NaN     NaN    NaN  NaN  
Alan Gravois  Nascent Letter       NaN  NaN  NaN  NaN     NaN    NaN  NaN  
Albert Fadden Terrestrial Chair    NaN  NaN  NaN  NaN     NaN    NaN  NaN  
Alberto Gumm  Terrestrial Chair    NaN  NaN  NaN  NaN     NaN    NaN  NaN

In [44]:
print(*gasto_sem_compra_concluida.index.get_level_values(0).unique(), sep='\n')

Adelina Thews
Alan Gravois
Albert Fadden
Alberto Gumm
Allen Bylsma
Allison Magill
Amanda Jones
Andrea Galyen
Andrew Hoff
Anna Hines
Anthony King
Anthony Nokes
Arthur Steiner
Belen Bradley
Benjamin Mcdonald
Benny Boisen
Beth Evans
Betty Lynch
Betty Moore
Beverly Redfern
Blanche Knox
Bobby Rick
Brandy Barnes
Brian Mann
Brittney Birkey
Bruce Hicks
Bryan Rayford
Caleb Munselle
Candy Killough
Carl Weber
Carol Eash
Carol Meeker
Cassandra Gonsalves
Catalina Berry
Catherine Loos
Charles Hershenson
Chelsea Banter
Cheryl West
Chester Chavez
Chris Wright
Christopher Mueller
Clarence Sharp
Courtney Chapman
Craig Wrisley
Cynthia Daves
Daniel Markell
Daniel Puckett
Danielle Gill
Darrel Allmand
David Pendergraph
Debra Howard
Denise Trapp
Dennis Miller
Derrick Collins
Donald Williams
Donna Marinas
Donna Wal
Dorothy Simmons
Earl Thoburn
Edna Martinez
Edward Dick
Edward Scott
Eileen Gonzalez
Elena Lane
Elizabeth Barela
Elizabeth Pietrowski
Emil Nord
Emilio Mccoy
Emily Reece
Fannie Anderson
Felicia Hallm

# 4. 
> Qual o dia com maior gasto?

In [45]:
dia_maior_gasto= join_all.groupby(['dia','mes','ano']).sum().sort_values(by="gastos", ascending=False)
dia_maior_gasto.head(1)

,,,gastos
dia,mes,ano,
3,1,2020,336343.24


In [46]:
dia_maior_gasto['gastos']
# dia com mais gastos:show da banda Symbolic Toy , dia 03

dia  mes  ano 
3    1    2020    336343.24
1    1    2020    335599.89
2    1    2020    289020.01
Name: gastos, dtype: float64

# 5. 
> Faça uma lista com os clientes que desistiram de comprar o ingresso com a AT, a
soma do valor que foi gasto durante os shows e quais shows eles desistiram de
comprar. Usar para resposta o esquema abaixo:

{

"nome": <str>,
"gastos": <float>,
"shows": [<str>]
    
}
   

In [47]:
#Desistiram de comprar ingresso com a AT :problema no pagamento e não concluido

pessoas_desistiram = join_all[(join_all['status'] == 'Nao Concluido') | (join_all['status'] == 'Problema no Pagamento')& ~(join_all['status'] == 'Concluido')]
pessoas_desistiram

,ano,dia,mes,nome,gastos,status,tipo,show
4,2020,1,1,Andrea Galyen,3467.13,Nao Concluido,Pista,Terrestrial Chair
5,2020,1,1,Andrea Galyen,837.73,Nao Concluido,Pista,Terrestrial Chair
9,2020,1,1,Benjamin Mcdonald,2309.81,Nao Concluido,VIP,Terrestrial Chair
12,2020,1,1,Blanche Knox,924.31,Nao Concluido,Pista Premium,Terrestrial Chair
20,2020,1,1,Charles Hershenson,163.44,Nao Concluido,VIP,Terrestrial Chair
...,...,...,...,...,...,...,...,...
545,2020,3,1,Tina Yee,238.00,Problema no Pagamento,VIP,Symbolic Toy
546,2020,3,1,Tina Yee,1467.34,Problema no Pagamento,VIP,Symbolic Toy
547,2020,3,1,Tina Yee,2230.42,Problema no Pagamento,VIP,Symbolic Toy
548,2020,3,1,Tina Yee,2663.77,Problema no Pagamento,VIP,Symbolic Toy
